# Training: SQL (Medium-users)
Welcome to the training notebook on using SQL.

This notebook is pitched at medium-users who perform more advanced querying operations to retrieve heavily-wrangled data from SQL.

# What will this session cover?
This session will show you how to do the following things in SQL:

1. Data matrix/tidy data principles
1. Advanced filtering of data
1. Tables vs. Views
1. Three types of temporary tables
     - Local temporary table
     - Common-table-expression
     - Global temporary table
1. Subqueries
1.  Ordering records according to their groups by a counter
1. Pivoting data from long to wide shape
1. Unpivoting data from wide to long shape
